In [1]:
!pip install ultralytics
!pip install -U ipywidgets
!pip install ffmpeg-python

   ---------------------------------------- 0.0/139.4 kB ? eta -:--:--
   -------- ------------------------------- 30.7/139.4 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 139.4/139.4 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/215.0 kB ? eta -:--:--
   -------------------------------------- - 204.8/215.0 kB 4.1 MB/s eta 0:00:01
   ---------------------------------------- 215.0/215.0 kB 3.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   - -------------------------------------- 0.1/2.3 MB 3.2 MB/s eta 0:00:01
   ---- ----------------------------------- 0.3/2.3 MB 3.2 MB/s eta 0:00:01
   -------- ------------------------------- 0.5/2.3 MB 3.7 MB/s eta 0:00:01
   ----------- ---------------------------- 0.7/2.3 MB 3.8 MB/s eta 0:00:01
   --------------- ------------------------ 0.9/2.3 MB 3.9 MB/s eta 0:00:01
   -------------------- ------------------- 1.2/2.3 MB 4.5 MB/s eta 0:00:01
   -----------

In [16]:
import os
import re

# Dictionary mapping class labels to their respective indices
class_label_mapping = {
    '31': '0',
    '43': '1',
    '37': '2',
    '36': '3',
    '34': '4',
    '47': '5',
    '46': '6',
    '33': '7',
    '32': '8',
    '44': '9',
    '42': '10',
    '38': '11',
    '45': '12',
    '41': '13',
    '35': '14',
    '48': '15'
}

# Path to the folder containing the .txt annotation files
input_folder = "C:/Users/thend/Desktop/Pratik/Lower_jaw/normalized_annotations"

# Path to the folder where modified files will be saved
output_folder = "C:/Users/thend/Desktop/Pratik/Lower_jaw/Labelmapped_annotations"

# Ensure output folder exists, create if not
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Iterate through each file in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".txt"):
        input_file_path = os.path.join(input_folder, filename)
        output_file_path = os.path.join(output_folder, filename)
        
        with open(input_file_path, 'r') as f_in:
            lines = f_in.readlines()
        
        with open(output_file_path, 'w') as f_out:
            for line in lines:
                # Extract class label from the beginning of the line
                match = re.match(r'^(\d+)', line)
                if match:
                    class_label = match.group(1)
                    # Replace class label with its index if it exists in the mapping
                    if class_label in class_label_mapping:
                        new_line = class_label_mapping[class_label] + line[len(class_label):]
                        f_out.write(new_line)
                    else:
                        f_out.write(line)
                else:
                    f_out.write(line)

print("Files processed successfully!")


Files processed successfully!


In [17]:
from ultralytics import YOLO
import glob
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import math
import yaml
import torch
import cv2
import random
from IPython.display import HTML, display, Image
from base64 import b64encode

In [18]:
import os
import random
import shutil

# Define input and output folders
images_folder = "C:/Users/thend/Desktop/Pratik/Lower_jaw/Images"
annotations_folder = "C:/Users/thend/Desktop/Pratik/Lower_jaw/Labelmapped_annotations"
training_images_folder = "C:/Users/thend/Desktop/Pratik/Lower_jaw/dataset_kag/train/images"
validation_images_folder = "C:/Users/thend/Desktop/Pratik/Lower_jaw/dataset_kag/val/images"
testing_images_folder = "C:/Users/thend/Desktop/Pratik/Lower_jaw/dataset_kag/test/images"
training_annotation_folder = "C:/Users/thend/Desktop/Pratik/Lower_jaw/dataset_kag/train/labels"
validation_annotation_folder = "C:/Users/thend/Desktop/Pratik/Lower_jaw/dataset_kag/val/labels"
testing_annotation_folder = "C:/Users/thend/Desktop/Pratik/Lower_jaw/dataset_kag/test/labels"

# Create output folders if they don't exist
output_folders = [training_images_folder, validation_images_folder, testing_images_folder,
                  training_annotation_folder, validation_annotation_folder, testing_annotation_folder]
for folder in output_folders:
    os.makedirs(folder, exist_ok=True)

# List all images and annotations
images = os.listdir(images_folder)
annotations = os.listdir(annotations_folder)

# Combine images and annotations into pairs
image_annotation_pairs = list(zip(images, annotations))

# Shuffle the pairs
random.shuffle(image_annotation_pairs)

# Unzip the shuffled pairs
shuffled_images, shuffled_annotations = zip(*image_annotation_pairs)

# Calculate split sizes
total_images = len(shuffled_images)
train_size = int(0.7 * total_images)
val_size = int(0.15 * total_images)

# Split images
train_images = shuffled_images[:train_size]
val_images = shuffled_images[train_size:train_size + val_size]
test_images = shuffled_images[train_size + val_size:]

# Copy images to respective folders
for img in train_images:
    shutil.copy(os.path.join(images_folder, img), training_images_folder)
for img in val_images:
    shutil.copy(os.path.join(images_folder, img), validation_images_folder)
for img in test_images:
    shutil.copy(os.path.join(images_folder, img), testing_images_folder)

# Split annotations
train_annotations = [img.replace('.JPG', '.txt') for img in train_images]
val_annotations = [img.replace('.JPG', '.txt') for img in val_images]
test_annotations = [img.replace('.JPG', '.txt') for img in test_images]

# Copy annotations to respective folders
for annotation in train_annotations:
    shutil.copy(os.path.join(annotations_folder, annotation), training_annotation_folder)
for annotation in val_annotations:
    shutil.copy(os.path.join(annotations_folder, annotation), validation_annotation_folder)
for annotation in test_annotations:
    shutil.copy(os.path.join(annotations_folder, annotation), testing_annotation_folder)

print("Data splitting and copying completed.")


Data splitting and copying completed.


In [19]:
model = YOLO('yolov8n-seg.pt')


In [20]:
#Creating yaml file

yolo_yaml = {'path': "C:/Users/thend/Desktop/Pratik/Lower_jaw/dataset_kag",
             'train': os.path.join("C:/Users/thend/Desktop/Pratik/Lower_jaw/dataset_kag", 'train/images'),  
             'val': os.path.join("C:/Users/thend/Desktop/Pratik/Lower_jaw/dataset_kag", 'val/images'),
             'test': os.path.join("C:/Users/thend/Desktop/Pratik/Lower_jaw/dataset_kag", 'test/images'), 

            #number of classes
            'nc': 16,

            # Classes 
            'names' : {
              0: '31',
              1: '43',
              2: '37',
              3: '36',
              4: '34',
              5: '47',
              6: '46',
              7: '33',
              8: '32',
              9: '44',
              10: '42',
              11: '38',
              12: '45',
              13: '41',
              14: '35',
              15: '48'}
            }
    
with open(os.path.join('C:/Users/thend/Desktop/Pratik/Lower_jaw/', 'kagyaml.yaml'), 'w+') as file:
    yaml.dump(yolo_yaml, file)

In [21]:
model.train(data='C:/Users/thend/Desktop/Pratik/Lower_jaw/kagyaml.yaml', epochs=10, batch=32, imgsz=640, name='Initial training with yolov8')

New https://pypi.org/project/ultralytics/8.1.47 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.44  Python-3.12.1 torch-2.2.0 CPU (Intel Core(TM) i5-1035G1 1.00GHz)
engine\trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=C:/Users/thend/Desktop/Pratik/Lower_jaw/kagyaml.yaml, epochs=10, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=Initial training with yolov83, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=

train: Scanning C:\Users\thend\Desktop\Pratik\Lower_jaw\dataset_kag\train\labels... 350 images, 0 backgrounds, 0 corrupt: 100%|██████████| 350/350 [00:01<00:00, 193.25it/s]

train: New cache created: C:\Users\thend\Desktop\Pratik\Lower_jaw\dataset_kag\train\labels.cache



val: Scanning C:\Users\thend\Desktop\Pratik\Lower_jaw\dataset_kag\val\labels... 75 images, 0 backgrounds, 0 corrupt: 100%|██████████| 75/75 [00:00<00:00, 238.09it/s]

val: New cache created: C:\Users\thend\Desktop\Pratik\Lower_jaw\dataset_kag\val\labels.cache


Plotting labels to runs\segment\Initial training with yolov83\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.0005, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\segment\Initial training with yolov83
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       1/10         0G     0.7293       1.97      4.563     0.9942        450        640:  64%|██████▎   | 7/11 [23:39<14:13, 213.49s/it]